In [1]:
import pandas as pd

data = pd.read_csv('data/train.csv', index_col = 0, dtype={'multiple_full_time_jobs': 'boolean', 'combined_multiple_jobs': 'boolean'}).reset_index(drop=True)
data = data.fillna(True)
data = data.sample(frac=0.1)
data['multiple_full_time_jobs'] = data['multiple_full_time_jobs'].astype(int)
data['combined_multiple_jobs'] = data['combined_multiple_jobs'].astype(int)
data.HIREDT = data.HIREDT.apply(pd.to_datetime, format = "%m/%d/%y")
data['daysexperience'] = (pd.to_datetime('10/7/23', format = "%m/%d/%y") - data.HIREDT ).dt.days
data_num = ['RATE', 'HRSWKD', 'daysexperience']
data_drop = ['AGY', 'HIREDT']#, 'multiple_full_time_jobs', 'combined_multiple_jobs']
data.drop(data_drop, axis=1, inplace=True)

In [2]:
data

,NAME,MI,JOBCLASS,JC.TITLE,RACE,SEX,EMPTYPE,RATE,HRSWKD,ANNUAL,STATENUM,multiple_full_time_jobs,combined_multiple_jobs,daysexperience
62888,DEPARTMENT OF FAMILY AND PROTECTIVE SERVICES ...,,5026,CHILD PROTECTIVE SVCS SPEC IV ...,WHITE,MALE,CRF - CLASSIFIED REGULAR FULL-TIME,0.0,40.0,60959.64,961608,1,1,2777
99761,TEXAS DEPARTMENT OF CRIMINAL JUSTICE ...,R,4504,CORREC OFFICER IV ...,BLACK,MALE,CRF - CLASSIFIED REGULAR FULL-TIME,0.0,40.0,45322.92,1457848,1,1,558
83277,HEALTH AND HUMAN SERVICES COMMISSION ...,,5152,PSYCHIATRIC NURSING ASST II ...,WHITE,FEMALE,CRF - CLASSIFIED REGULAR FULL-TIME,0.0,40.0,36205.32,1236953,1,1,874
99918,WATER DEVELOPMENT BOARD ...,L,1623,DIRECTOR IV ...,WHITE,MALE,CRF - CLASSIFIED REGULAR FULL-TIME,0.0,40.0,110000.04,1459561,1,1,554
10446,TEXAS JUVENILE JUSTICE DEPARTMENT ...,R,4524,JUVENILE CORREC OFFCR V ...,WHITE,FEMALE,CRF - CLASSIFIED REGULAR FULL-TIME,0.0,40.0,52037.40,50558,1,1,2715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28511,HEALTH AND HUMAN SERVICES COMMISSION ...,M,5123,DIRECT SUPPORT PROF III ...,HISPANIC,FEMALE,CRF - CLASSIFIED REGULAR FULL-TIME,0.0,40.0,40951.08,138914,1,1,2227
13513,TEXAS WORKFORCE COMMISSION ...,,5702,H/SRVC SPEC III ...,ASIAN,FEMALE,CRF - CLASSIFIED REGULAR FULL-TIME,0.0,40.0,41672.40,65519,1,1,2592
20142,OFFICE OF INJURED EMPLOYEE COUNSEL ...,A,3663,OMBUDSMAN III ...,HISPANIC,FEMALE,CRF - CLASSIFIED REGULAR FULL-TIME,0.0,40.0,51657.48,98302,1,1,5822
21868,HEALTH AND HUMAN SERVICES COMMISSION ...,,5052,REHAB THERAPY TECHNICIAN III ...,HISPANIC,MALE,CRF - CLASSIFIED REGULAR FULL-TIME,0.0,40.0,28085.88,106602,1,1,2227


In [3]:
categorical_columns = ['NAME', 'MI', 'JOBCLASS', 'JC.TITLE', 'RACE', 'SEX', 'EMPTYPE', 'STATENUM', 'multiple_full_time_jobs', 'combined_multiple_jobs']

from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
import numpy as np
regressor = DecisionTreeRegressor(max_depth=1)

categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value",
                                          unknown_value=-1)

preprocessor = ColumnTransformer([
    ('cat_preprocessor', categorical_preprocessor, categorical_columns)],
    remainder='passthrough', sparse_threshold=0)

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline

model = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier",
     GradientBoostingRegressor(random_state=42, max_leaf_nodes=4))])

from sklearn.model_selection import train_test_split
X = data.drop('ANNUAL', axis=1)
y = data.ANNUAL

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=0)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [4]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for the Decision Tree Regressor
param_grid = {
    'regressor__max_depth': range(1, 20),
    'regressor__min_samples_split': range(2, 10),
    'regressor__min_samples_leaf': range(1, 10),
}

# Instantiate the GridSearchCV object
grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='r2', n_jobs=-1)

# Fit the GridSearchCV object
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best parameters: {best_params}")
print(f"Best R-squared score: {best_score:.2f}")

NameError: name 'pipe' is not defined

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_pred, y_test)

In [ ]:
import pandas as pd

data = pd.read_csv('data/train.csv', index_col = 0, dtype={'multiple_full_time_jobs': 'boolean', 'combined_multiple_jobs': 'boolean'}).reset_index(drop=True)
# Display the first few rows of the dataset
top_rows_str = data.head().to_string()

# Display the summary statistics of the dataset
summary_statistics_str = data.describe().to_string()

# Check the correlation between the features and the target variable
correlations = data.corr()
correlations_str = correlations.to_string()

output_str = f"First few rows of the dataset:\n{top_rows_str}\n\nSummary statistics of the dataset:\n{summary_statistics_str}\n\nCorrelation matrix:\n{correlations_str}"
output_str

In [7]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder

# Load the data
data = pd.read_csv('data/train.csv', index_col = 0, dtype={'multiple_full_time_jobs': 'boolean', 'combined_multiple_jobs': 'boolean'}).reset_index(drop=True)
data = data.fillna(True)
data = data.sample(frac=0.1)
data['multiple_full_time_jobs'] = data['multiple_full_time_jobs'].astype(int)
data['combined_multiple_jobs'] = data['combined_multiple_jobs'].astype(int)
data.HIREDT = data.HIREDT.apply(pd.to_datetime, format = "%m/%d/%y")
data['daysexperience'] = (pd.to_datetime('10/7/23', format = "%m/%d/%y") - data.HIREDT ).dt.days
data_num = ['RATE', 'HRSWKD', 'daysexperience']
data_drop = ['AGY', 'HIREDT']#, 'multiple_full_time_jobs', 'combined_multiple_jobs']
data.drop(data_drop, axis=1, inplace=True)

In [8]:
# Define the categorical columns and preprocessor
categorical_columns = ['NAME', 'MI', 'JOBCLASS', 'JC.TITLE', 'RACE', 'SEX', 'EMPTYPE', 'STATENUM', 'multiple_full_time_jobs', 'combined_multiple_jobs']
categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

# Define the preprocessor pipeline
preprocessor = ColumnTransformer([
    ('cat_preprocessor', categorical_preprocessor, categorical_columns)],
    remainder='passthrough', sparse_threshold=0)

# Define the regressor pipeline with default hyperparameters
regressor = DecisionTreeRegressor(max_depth=1)
model = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", GradientBoostingRegressor(random_state=42, max_leaf_nodes=4))
])

# Split the data into train and test sets
X = data.drop('ANNUAL', axis=1)
y = data.ANNUAL
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3)

# Define the hyperparameters to search over
hyperparameters = {
    'classifier__learning_rate': np.logspace(-3, 0, num=40),
    'classifier__n_estimators': [400, 800, 1600, 2000],
    'classifier__max_depth': [5, 6, 7, 8, 9, 10],
    'classifier__max_features': [1.0, 'sqrt', 'log2', None]
}

# Use RandomizedSearchCV to find the best hyperparameters
search = GridSearchCV(model, hyperparameters, cv=2, n_jobs=-1, verbose=2)
# search = RandomizedSearchCV(model, hyperparameters, cv=5, n_iter=20, n_jobs=-1, verbose=1, random_state=42)
search.fit(X_train, y_train)

# Print the best hyperparameters and score
print("Best hyperparameters: ", search.best_params_)
print("Best score: ", search.best_score_)

# Evaluate the performance of the model on the test set
y_pred = search.predict(X_test)
from sklearn.metrics import r2_score
print("R2 score on test set: ", r2_score(y_test, y_pred))

Fitting 2 folds for each of 3840 candidates, totalling 7680 fits
[CV] END classifier__learning_rate=0.001, classifier__max_depth=5, classifier__max_features=1.0, classifier__n_estimators=1600; total time=   7.2s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=5, classifier__max_features=None, classifier__n_estimators=400; total time=   1.8s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=5, classifier__max_features=None, classifier__n_estimators=2000; total time=   8.4s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=6, classifier__max_features=sqrt, classifier__n_estimators=2000; total time=   2.4s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=6, classifier__max_features=log2, classifier__n_estimators=2000; total time=   2.5s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=6, classifier__max_features=None, classifier__n_estimators=2000; total time=   8.3s
[CV] END classifier__learning_rate=0.001, classif

[CV] END classifier__learning_rate=0.001, classifier__max_depth=5, classifier__max_features=1.0, classifier__n_estimators=800; total time=   3.5s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=5, classifier__max_features=sqrt, classifier__n_estimators=2000; total time=   2.5s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=5, classifier__max_features=log2, classifier__n_estimators=1600; total time=   2.0s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=5, classifier__max_features=None, classifier__n_estimators=800; total time=   3.5s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=6, classifier__max_features=1.0, classifier__n_estimators=1600; total time=   6.6s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=6, classifier__max_features=log2, classifier__n_estimators=400; total time=   0.5s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=6, classifier__max_features=log2, classifier__n_estim

[CV] END classifier__learning_rate=0.001, classifier__max_depth=5, classifier__max_features=1.0, classifier__n_estimators=400; total time=   1.8s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=5, classifier__max_features=sqrt, classifier__n_estimators=400; total time=   0.5s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=5, classifier__max_features=sqrt, classifier__n_estimators=800; total time=   1.0s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=5, classifier__max_features=sqrt, classifier__n_estimators=1600; total time=   1.9s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=5, classifier__max_features=log2, classifier__n_estimators=400; total time=   0.5s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=5, classifier__max_features=log2, classifier__n_estimators=800; total time=   1.0s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=5, classifier__max_features=log2, classifier__n_estima

[CV] END classifier__learning_rate=0.001, classifier__max_depth=5, classifier__max_features=1.0, classifier__n_estimators=400; total time=   1.8s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=5, classifier__max_features=sqrt, classifier__n_estimators=400; total time=   0.5s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=5, classifier__max_features=sqrt, classifier__n_estimators=800; total time=   1.0s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=5, classifier__max_features=sqrt, classifier__n_estimators=1600; total time=   2.0s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=5, classifier__max_features=log2, classifier__n_estimators=400; total time=   0.5s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=5, classifier__max_features=log2, classifier__n_estimators=800; total time=   1.1s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=5, classifier__max_features=None, classifier__n_estima

[CV] END classifier__learning_rate=0.001, classifier__max_depth=5, classifier__max_features=1.0, classifier__n_estimators=2000; total time=   8.5s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=5, classifier__max_features=None, classifier__n_estimators=1600; total time=   6.7s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=6, classifier__max_features=sqrt, classifier__n_estimators=400; total time=   0.5s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=6, classifier__max_features=sqrt, classifier__n_estimators=400; total time=   0.5s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=6, classifier__max_features=sqrt, classifier__n_estimators=800; total time=   1.0s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=6, classifier__max_features=sqrt, classifier__n_estimators=1600; total time=   2.0s
[CV] END classifier__learning_rate=0.001, classifier__max_depth=6, classifier__max_features=log2, classifier__n_esti

...

[CV] END classifier__learning_rate=0.41246263829013524, classifier__max_depth=9, classifier__max_features=None, classifier__n_estimators=400; total time=   2.1s
[CV] END classifier__learning_rate=0.41246263829013524, classifier__max_depth=9, classifier__max_features=None, classifier__n_estimators=2000; total time=  10.4s
[CV] END classifier__learning_rate=0.41246263829013524, classifier__max_depth=10, classifier__max_features=log2, classifier__n_estimators=2000; total time=   3.0s
[CV] END classifier__learning_rate=0.49238826317067413, classifier__max_depth=5, classifier__max_features=1.0, classifier__n_estimators=400; total time=   2.2s
[CV] END classifier__learning_rate=0.49238826317067413, classifier__max_depth=5, classifier__max_features=1.0, classifier__n_estimators=1600; total time=   8.1s
[CV] END classifier__learning_rate=0.49238826317067413, classifier__max_depth=5, classifier__max_features=None, classifier__n_estimators=800; total time=   4.1s
[CV] END classifier__learning_ra

[CV] END classifier__learning_rate=0.8376776400682924, classifier__max_depth=6, classifier__max_features=sqrt, classifier__n_estimators=400; total time=   0.4s
[CV] END classifier__learning_rate=0.8376776400682924, classifier__max_depth=6, classifier__max_features=sqrt, classifier__n_estimators=800; total time=   0.8s
[CV] END classifier__learning_rate=0.8376776400682924, classifier__max_depth=6, classifier__max_features=sqrt, classifier__n_estimators=1600; total time=   1.7s
[CV] END classifier__learning_rate=0.8376776400682924, classifier__max_depth=6, classifier__max_features=log2, classifier__n_estimators=400; total time=   0.5s
[CV] END classifier__learning_rate=0.8376776400682924, classifier__max_depth=6, classifier__max_features=log2, classifier__n_estimators=800; total time=   0.9s
[CV] END classifier__learning_rate=0.8376776400682924, classifier__max_depth=6, classifier__max_features=log2, classifier__n_estimators=2000; total time=   2.2s
[CV] END classifier__learning_rate=0.8

[CV] END classifier__learning_rate=0.701703828670383, classifier__max_depth=7, classifier__max_features=sqrt, classifier__n_estimators=800; total time=   0.9s
[CV] END classifier__learning_rate=0.701703828670383, classifier__max_depth=7, classifier__max_features=sqrt, classifier__n_estimators=1600; total time=   1.7s
[CV] END classifier__learning_rate=0.701703828670383, classifier__max_depth=7, classifier__max_features=log2, classifier__n_estimators=400; total time=   0.5s
[CV] END classifier__learning_rate=0.701703828670383, classifier__max_depth=7, classifier__max_features=log2, classifier__n_estimators=800; total time=   0.9s
[CV] END classifier__learning_rate=0.701703828670383, classifier__max_depth=7, classifier__max_features=log2, classifier__n_estimators=2000; total time=   2.2s
[CV] END classifier__learning_rate=0.701703828670383, classifier__max_depth=7, classifier__max_features=None, classifier__n_estimators=2000; total time=   7.4s
[CV] END classifier__learning_rate=0.701703

[CV] END classifier__learning_rate=0.49238826317067413, classifier__max_depth=10, classifier__max_features=None, classifier__n_estimators=1600; total time=   5.9s
[CV] END classifier__learning_rate=0.5878016072274912, classifier__max_depth=5, classifier__max_features=sqrt, classifier__n_estimators=400; total time=   0.4s
[CV] END classifier__learning_rate=0.5878016072274912, classifier__max_depth=5, classifier__max_features=sqrt, classifier__n_estimators=400; total time=   0.4s
[CV] END classifier__learning_rate=0.5878016072274912, classifier__max_depth=5, classifier__max_features=sqrt, classifier__n_estimators=800; total time=   0.8s
[CV] END classifier__learning_rate=0.5878016072274912, classifier__max_depth=5, classifier__max_features=sqrt, classifier__n_estimators=1600; total time=   1.7s
[CV] END classifier__learning_rate=0.5878016072274912, classifier__max_depth=5, classifier__max_features=log2, classifier__n_estimators=800; total time=   0.9s
[CV] END classifier__learning_rate=0

[CV] END classifier__learning_rate=0.41246263829013524, classifier__max_depth=5, classifier__max_features=None, classifier__n_estimators=400; total time=   2.2s
[CV] END classifier__learning_rate=0.41246263829013524, classifier__max_depth=6, classifier__max_features=1.0, classifier__n_estimators=400; total time=   2.1s
[CV] END classifier__learning_rate=0.41246263829013524, classifier__max_depth=6, classifier__max_features=1.0, classifier__n_estimators=1600; total time=   8.4s
[CV] END classifier__learning_rate=0.41246263829013524, classifier__max_depth=6, classifier__max_features=None, classifier__n_estimators=400; total time=   2.2s
[CV] END classifier__learning_rate=0.41246263829013524, classifier__max_depth=7, classifier__max_features=1.0, classifier__n_estimators=400; total time=   2.1s
[CV] END classifier__learning_rate=0.41246263829013524, classifier__max_depth=7, classifier__max_features=1.0, classifier__n_estimators=1600; total time=   8.2s
[CV] END classifier__learning_rate=0